## Análisis de datos - Clase 7
---

## Taller práctico parte 2 - preparación completa de un dataset para entrenamiento de modelos de IA
---


Vamos a realizar una actividad con un dataset con los datos del Servicio Meteorológico Nacional Argentino que utilizamos en el taller práctico 1.

A modo de repaso, el dataset contiene estadísticas climáticas normales (período 1991-2020) y temperaturas máximas y mínimas registradas durante el último año (365 días).

Fuente: https://www.smn.gob.ar/descarga-de-datos

En esta segunda parte del taller, vamos a preparar el dataset para que pueda ser utilizado en el entrenamiento de modelos de IA para resolver un determinado problema de ML.


**Resumen de descubrimientos del taller 1:**

**Variables**:

* estación, provincia, latitud, longitud, altitud, número y NroOACI (estos dos últimos identifican las estaciones)
* mes
* temperatura media, máxima y mínima
* humedad relativa
* velocidad del viento
* nubosidad total (en meteorología, la nubosidad se mide en octas (0, despejado - 8 completamente cubierto), que representan la fracción del cielo cubierta por nubes)
* precipitación
* frecuencia de días con precipitación superior a 0,1 mm.

**Dimensiones**: (1176, 16)

**Tipos de datos**:
* Variables numéricas:  'altura', 'LAT_decimal','LON_decimal', 'prec_sup_1mm', 'humedad', 'nubosidad', 'prec_mm', 'temp', 'temp_max', 'temp_min', 'viento'
* Variables categóricas: 'estacion', 'provincia','numero', 'NroOACI', 'mes_txt'

**Generales**:
* No hay variables ordinales.
* Hay 98 observaciones (filas) para cada mes del año.
* Hay más estaciones en la provincia de Buenos Aires que en cualquier otra provincia.

**Datos faltantes (resumen)**:
| Variable       | Nro Faltantes | % Faltantes | Causa | Comentarios |
|----------------|---------------|-------------|-------|-------------|
| viento         | 468           | 39.80%      | Inconcluso  | En este caso podemos considerarla MAR para el ejercicio, pero requiere investigación. Potencialmente no es un faltante sino que ciertas estaciones (94) no miden viento, o no lo reportan. |
| prec_sup_1mm   | 115           | 9.78%       | MAR   | prec_mm y prec_sum_1mm se mueven juntas.Asumimos MAR, pero requiere investigación. Solo falta para las bases de la Antártida y Santa Cruz.                |
| prec_mm        | 115           | 9.78%       | MAR   | prec_mm y prec_sum_1mm se mueven juntas. Asumimos MAR, pero requiere investigación. Solo falta para las bases de la Antártida y Santa Cruz |
| humedad        | 66            | 5.61%       | MAR   | Parecen estar asociados a ciertas estaciones y a cierta época del año.|
| temp_min       | 52            | 4.42%       | MAR   |Parecen ocurrir solo en ciertas estaciones meteorológicas |
| NroOACI        | 36            | 3.06%       | No son faltantes  | Corresponden a 3 estaciones sin OACI asignado|
| provincia      | 24            | 2.04%       | N/A  |Se pueden completar (se tiene el nombre de la estación meteorológica) |
| altura         | 24            | 2.04%       | N/A |Se pueden completar (se tiene el nombre de la estación meteorológica) |
| numero         | 24            | 2.04%       | N/A |Se pueden completar (se tiene el nombre de la estación meteorológica) |
| LAT_decimal    | 24            | 2.04%       | N/A |Se pueden completar (se tiene el nombre de la estación meteorológica) |
| LON_decimal    | 24            | 2.04%       | N/A |Se pueden completar (se tiene el nombre de la estación meteorológica) |
| temp_max       | 18            | 1.53%       | MAR  |Asociados a ciertas estaciones meteorológicas|
| nubosidad      | 10            | 0.85%       | MAR  |Asociados a ciertas estaciones meteorológicas y meses del año|
| temp           | 9             | 0.77%       | MAR  |Asociados a ciertas estaciones meteorológicas y meses del año|

**Outliers**:

1) Estaciones meteorológicas

- Los valores atípicos en altura no son errores. Corresponden a estaciones meteorológicas localizadas en zonas de cordillera o precordillera.
- Los valores de latitud y longitud extremadamente alejadas del resto también se consideran datos válidos ya que corresponden a estaciones de la Antártida.

2) Temperaturas

- Las temperaturas extremas negativas corresponden a estaciones meteorológicas antárticas, por lo que consideran válidas.

3) Nubosidad

- La variable nubosidad muestra valores atípicos mayores a 5,5. Estos números representan un cielo que en promedio está mayormente nublado a completamente cubierto. Se observa para estaciones de la Antártida, lo que hace pensar que en este caso también son valores correctos.

4) Otras variables

- La cantidad de valores atípicos es despreciable.

En general en todo el set de datos, los valores atípicos se presentan principalmente en las bases antárticas y en todos los casos se puede concluir que son valores legítimos.


**Para más detalles del EDA y exploración, ver notebook del taller práctico 1.**

---
El **problema de ML propuesto** es predecir la temperatura promedio mensual (problema de regresión), dada una determinada Latitud, Longitud, altura y otros datos climáticos.

---

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from statsmodels.imputation import mice
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest,SelectFpr, chi2, f_classif, mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

---
### Importar los datos
---

In [2]:
smn_historico = pd.read_csv('../datasets/smn/smn_historico.csv', na_values=['S/D'])

In [3]:
# Completamos datos faltantes que se podían sacar de Internet (https://heladas.agro.uba.ar/estaciones.htm)
smn_historico.loc[smn_historico['estacion'] == 'EL TREBOL', 'provincia'] = 'SANTA FE'
smn_historico.loc[smn_historico['estacion'] == 'EL TREBOL', 'LAT_decimal'] = -32.202
smn_historico.loc[smn_historico['estacion'] == 'EL TREBOL', 'LON_decimal'] = -61.7
smn_historico.loc[smn_historico['estacion'] == 'EL TREBOL', 'altura'] = 96
smn_historico.loc[smn_historico['estacion'] == 'CORONEL PRINGLES AERO', 'provincia'] = 'BUENOS AIRES'
smn_historico.loc[smn_historico['estacion'] == 'CORONEL PRINGLES AERO', 'LAT_decimal'] = -38.02
smn_historico.loc[smn_historico['estacion'] == 'CORONEL PRINGLES AERO', 'LON_decimal'] = -61.33
smn_historico.loc[smn_historico['estacion'] == 'CORONEL PRINGLES AERO', 'altura'] = 247


In [4]:
smn_historico.shape

(1176, 16)

In [5]:
# Visualizar primeras filas
smn_historico.head()

,estacion,provincia,altura,numero,NroOACI,LAT_decimal,LON_decimal,mes_txt,prec_sup_1mm,humedad,nubosidad,prec_mm,temp,temp_max,temp_min,viento
0,AEROPARQUE AERO,CAPITAL FEDERAL,6.0,87582.0,SABE,-34.55,-58.416667,Ene,6.7,67.0,3.1,117.5,24.5,28.4,20.8,16.8
1,AEROPARQUE AERO,CAPITAL FEDERAL,6.0,87582.0,SABE,-34.55,-58.416667,Feb,6.0,69.8,3.2,112.3,23.7,27.3,20.2,15.8
2,AEROPARQUE AERO,CAPITAL FEDERAL,6.0,87582.0,SABE,-34.55,-58.416667,Mar,5.9,71.3,3.1,111.8,22.0,25.5,18.8,14.9
3,AEROPARQUE AERO,CAPITAL FEDERAL,6.0,87582.0,SABE,-34.55,-58.416667,Abr,6.6,73.6,3.5,108.3,18.5,22.0,15.3,13.9
4,AEROPARQUE AERO,CAPITAL FEDERAL,6.0,87582.0,SABE,-34.55,-58.416667,May,5.0,76.4,4.0,83.3,15.2,18.4,12.3,12.9


---
### Tratamiento de datos faltantes
---

¿Qué estrategias aplicarían para manejar los valores faltantes del dataset? ¿Eliminar, imputar? ¿Qué métodos usarían? Justifiquen su elección.


---
### Tratamiento de outliers
---

¿Cómo abordarían la presencia de outliers? ¿Conservarlos, eliminarlos, imputarlos? ¿Qué técnicas usarían? Expliquen ventajas y riesgos.


---
### Codificación
---

¿Qué técnicas de codificación usarían para las variables categóricas? Justifiquen su decisión.

---
### Escalamiento
---

¿Aplicarían escalamiento a las variables numéricas? ¿Cuál método elegirían? ¿Por qué harían esto?